In [1]:
import sys
import os
current_dir = os.getcwd()  
project_root = os.path.dirname(current_dir)  
sys.path.insert(0, project_root) 
from pathlib import Path
import warnings
import pandas as pd
from pycaret.regression import *
from src.config.config import Config
from src.data.data_loader import DataLoader
from src.models.model_evaluator import ModelEvaluator

def train_models(train: pd.DataFrame,val: pd.DataFrame,target_column: str):
    """训练模型流程
    Args:
        data_path: 可选的数据文件路径
    """
    model_evaluator.setup_experiment(
        train_size=Config.MODEL_CONFIG['train_size'],
        train_data=train,
        val_data = val,
        target_column = target_column,
        categorical_features = Config.FEATURE_CONFIG['categorical_columns']
    )

    # 训练模型并返回最佳模型字典
    return model_evaluator.train_top_models(n_models=Config.MODEL_CONFIG['n_top_models'])

def tune(models: dict):
    """模型调优和集成"""
    # 对top模型进行调优
    tuned_models = model_evaluator.tune_models(models)

    return tuned_models

def ensemble(models: dict):
    """模型集成"""
    # 对top模型进行集成
    blended_model = model_evaluator.ensemble_models(models)

    return blended_model    
    # 模型
    
def save_models(models: dict):

    model_evaluator.saved_models(models)

In [2]:
# 示例用法
df = pd.read_csv(Path(Config.PATH_CONFIG['features_dir'])  / 'training_data.csv')
data_loader = DataLoader()
model_evaluator = ModelEvaluator()

# 划分数据集
train_data, val_data, country_test_data = data_loader.split_data_by_countries(
    df,
    train_size=Config.DATA_CONFIG['country_train_size'],
    val_size=Config.DATA_CONFIG['country_val_size'],
    random_state=Config.DATA_CONFIG['random_state']
)

train_data, time_test_data = data_loader.split_data_by_time(train_data,
    test_size=Config.DATA_CONFIG['time_test_size'])

# 数据分析
print('\n训练集统计信息:')
data_loader.analyze_datasets(train_data)
print('\n验证集统计信息:')
data_loader.analyze_datasets(val_data)
print('\n外部国家测试集统计信息:')
data_loader.analyze_datasets(country_test_data)
print('\n外部时间测试集统计信息:')
data_loader.analyze_datasets(time_test_data)
#合并验证集和外部国家测试集作为测试集
test_data = pd.concat([val_data, country_test_data])
print('\n合并，作为测试集统计信息:')
data_loader.analyze_datasets(test_data)


训练集统计信息:
总数据条数: 1399
国家总数: 57
包含的国家: Albania, Argentina, Austria, Bangladesh, Belgium, Bosnia and Herzegovina, Brazil, Bulgaria, Canada, China, Colombia, Congo, Dem. Rep., Croatia, Cyprus, Czechia, Denmark, Estonia, Ethiopia, Greece, Hungary, Iceland, Indonesia, Iran, Islamic Rep., Iraq, Ireland, Italy, Japan, Korea, Rep., Latvia, Lithuania, Luxembourg, Malaysia, Malta, Mexico, Montenegro, Morocco, Netherlands, Nigeria, North Macedonia, Norway, Peru, Philippines, Poland, Romania, Russian Federation, Saudi Arabia, Serbia, Slovak Republic, Slovenia, South Africa, Spain, Sweden, Switzerland, Tanzania, Turkiye, Uganda, United Kingdom

特征数量: 24
特征列表: MSW, MSW_log, development_stage, gdp_10y_ma, gdp_5y_ma, gdp_acceleration, gdp_growth_rate, gdp_per_capita_growth, gdp_per_capita_ma, gdp_pop_interaction, gdp_trend, pop_density_trend, pop_growth_rate, pop_trend, region_avg_gdp, region_gdp_per_capita, similar_gdp_growth, stage_avg_gdp_growth, stage_weight, weighted_gdp, weighted_pop, year_trend

In [3]:
train_data.to_csv(os.path.join(Config.PATH_CONFIG['data_dir'], 'train.csv'), index=False)
test_data.to_csv(os.path.join(Config.PATH_CONFIG['data_dir'], 'test.csv'), index=False)
time_test_data.to_csv(os.path.join(Config.PATH_CONFIG['data_dir'], 'time_test_data.csv'), index=False)

# 排除不需要的列
feature_cols = [col for col in train_data.columns 
                if col not in [Config.DATA_CONFIG['target_column'], 'Country Name', 'Year','Region', 'Income Group']]

train = train_data[feature_cols]
test = test_data[feature_cols]
time_test_data = time_test_data[feature_cols] 

target_column = Config.DATA_CONFIG['target_column']
method = Config.FEATURE_CONFIG['target_transform_method']
transformed_column = f'{target_column}_{method}'


In [4]:
# 训练模型
models = train_models(train,test,transformed_column)

,Description,Value
0,Session id,888
1,Target,MSW_log
2,Target type,Regression
3,Original data shape,"(1769, 26)"
4,Transformed data shape,"(1769, 30)"
5,Transformed train set shape,"(1399, 30)"
6,Transformed test set shape,"(370, 30)"
7,Numeric features,24
8,Categorical features,1
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.2901,0.1607,0.3824,0.8899,0.0242,0.0193,0.0380
omp,Orthogonal Matching Pursuit,0.2951,0.1620,0.3780,0.8799,0.0237,0.0195,0.3380
gbr,Gradient Boosting Regressor,0.3253,0.1864,0.4185,0.8848,0.0264,0.0216,0.1200
lightgbm,Light Gradient Boosting Machine,0.3433,0.2236,0.4536,0.8606,0.0283,0.0226,0.0960
rf,Random Forest Regressor,0.3532,0.2190,0.4513,0.8587,0.0282,0.0233,0.0680
xgboost,Extreme Gradient Boosting,0.3526,0.2196,0.4537,0.8654,0.0285,0.0234,0.0900
catboost,CatBoost Regressor,0.3448,0.2825,0.4724,0.7877,0.0304,0.0236,0.4840
ada,AdaBoost Regressor,0.3555,0.2273,0.4646,0.8442,0.0293,0.0236,0.0380
huber,Huber Regressor,0.3680,0.2781,0.4797,0.7792,0.0294,0.0237,0.0120
br,Bayesian Ridge,0.3839,0.3041,0.5036,0.7700,0.0309,0.0248,0.0120


In [5]:
# 调优
tune_models = tune(models)

当前调优模型: et , 结果如下:


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4838,0.4093,0.6397,0.6233,0.0423,0.0342
1,0.3144,0.2202,0.4693,0.9343,0.0320,0.0218
2,0.1730,0.0602,0.2454,0.9795,0.0150,0.0114
3,0.2329,0.0792,0.2814,0.9222,0.0174,0.0153
4,0.2017,0.0702,0.2650,0.9363,0.0151,0.0122
Mean,0.2811,0.1678,0.3802,0.8791,0.0244,0.0190
Std,0.1118,0.1342,0.1526,0.1294,0.0110,0.0084


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.3172,0.2107,0.4590,0.8656,0.0240,0.0178


当前调优模型: omp , 结果如下:


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3619,0.1815,0.4260,0.8329,0.0276,0.0250
1,0.3295,0.1669,0.4085,0.9502,0.0263,0.0218
2,0.1766,0.0464,0.2155,0.9842,0.0141,0.0122
3,0.1777,0.0457,0.2138,0.9551,0.0129,0.0115
4,0.3099,0.1229,0.3506,0.8885,0.0205,0.0190
Mean,0.2711,0.1127,0.3229,0.9222,0.0203,0.0179
Std,0.0785,0.0577,0.0918,0.0544,0.0060,0.0053


...

[LightGBM] [Warning] feature_fraction is set=0.6079388442943392, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6079388442943392
[LightGBM] [Warning] bagging_fraction is set=0.4456980213134, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4456980213134
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.3113,0.1866,0.4320,0.8809,0.0227,0.0175


[LightGBM] [Warning] feature_fraction is set=0.6079388442943392, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6079388442943392
[LightGBM] [Warning] bagging_fraction is set=0.4456980213134, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4456980213134
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
当前调优模型: rf , 结果如下:


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4900,0.3791,0.6157,0.6510,0.0397,0.0337
1,0.3401,0.2254,0.4747,0.9328,0.0323,0.0235
2,0.3339,0.1948,0.4414,0.9338,0.0269,0.0218
3,0.2889,0.1360,0.3688,0.8664,0.0228,0.0190
4,0.1905,0.0580,0.2408,0.9474,0.0138,0.0116
Mean,0.3287,0.1987,0.4283,0.8663,0.0271,0.0219
Std,0.0968,0.1068,0.1234,0.1113,0.0088,0.0071


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.3154,0.1853,0.4304,0.8818,0.0227,0.0178


In [6]:
暂停

NameError: name '暂停' is not defined

In [7]:
# 定义需要包含的模型简称集合
include_models = {'rf','gbr','lightgbm','et'}

# 生成过滤后的有序字典
model_dict = {
    k: v for k, v in tune_models.items()
    if k in include_models
}

In [8]:
# 集成
models = ensemble(model_dict)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4879,0.3842,0.6199,0.6463,0.0404,0.0339
1,0.2876,0.2153,0.4640,0.9358,0.0324,0.0204
2,0.2399,0.1028,0.3206,0.9651,0.0193,0.0157
3,0.2660,0.1110,0.3331,0.8910,0.0206,0.0174
4,0.1995,0.0649,0.2547,0.9412,0.0146,0.0122
Mean,0.2962,0.1756,0.3985,0.8759,0.0254,0.0199
Std,0.1003,0.1156,0.1299,0.1172,0.0095,0.0075


[LightGBM] [Warning] feature_fraction is set=0.6079388442943392, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6079388442943392
[LightGBM] [Warning] bagging_fraction is set=0.4456980213134, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4456980213134
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Extra Trees Regressor,0.2811,0.1678,0.3802,0.8791,0.0244,0.0190,0.4840
1,Gradient Boosting Regressor,0.3065,0.1746,0.4013,0.8762,0.0256,0.0206,0.4640
4,Voting Regressor,0.2962,0.1756,0.3985,0.8759,0.0254,0.0199,0.7060
3,Random Forest Regressor,0.3287,0.1987,0.4283,0.8663,0.0271,0.0219,0.4680
2,Light Gradient Boosting Machine,0.3339,0.2227,0.4508,0.8471,0.0287,0.0224,0.4700


In [9]:
save_model = save_models(models)

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
